In [3]:
#PREPROCESS_prepare_bubbleplot_ready_matrix

#input: DAP protein files (3 of them)

#input2: cytokine_annoation.tsv (this is a file that removed duplicated significant cytokines.
#cytokine_annoation.tsv have acquired using: "PREPROCESS_identify_cytokines_from_significant_proteins"
#and manual selection. Please look at the output of "PREPROCESS_identify_cytokines_from_significant_proteins" for better understanding.

import pandas as pd
import math

In [4]:
def categorize_pval(value):
    
    if value > 0.05:
        return (0)

    if value < 0.05 and value > 0.01:
        return (1)

    if value < 0.01 and value > 0.005:
        return (2)

    if value < 0.005:
        return (3)

In [6]:
annotation_file = "../../../analysis/statistics/cytokine_boxplots/cytokine_annoation.tsv"
dap_file_dir = "../../../analysis/statistics/linear_model/differential_abundance_logit"

annotation_df = pd.read_csv(annotation_file, sep="\t")
cytokine_list = list(annotation_df["protein"])
data_dict = {}

condition_list = ["cVSneg","cVSpos","negVSpos"]
for condition in condition_list:

    data_file = (f"{dap_file_dir}/proteomics.{condition}.tsv")
    data_df = pd.read_csv(data_file, sep="\t")

    filtered_data_df = data_df[data_df.iloc[:,0].isin(cytokine_list)].reset_index(drop=True)
    r, c = filtered_data_df.shape
    
    for i in range(r):
        protein = filtered_data_df.iloc[i, 0]
        cohenD = filtered_data_df["cohenD"][i]
        fc_case_control = filtered_data_df["fc_case_control"][i]
        pval = filtered_data_df["all_adj_pval"][i]

        data_dict[protein, condition, "cohenD"] = cohenD
        data_dict[protein, condition, "fc_case_control"] = fc_case_control
        data_dict[protein, condition, "pval"] = pval
        data_dict[protein, condition, "cat_pvalue"] = categorize_pval(pval)


In [7]:
output_file = "../../../analysis/statistics/cytokine_boxplots/cytokine.bubbleplot.ready.tsv"
output_txt = open(output_file, 'w')

output_txt.write("cytokine\tcondition\tcohenD\tfc_case_control\tpval\tcatpval\n")

for cytokine in cytokine_list:
    for condition in condition_list:
        
        output_txt.write(f"{cytokine}\t{condition}\t{data_dict[cytokine, condition, "cohenD"]}\t")
        output_txt.write(f"{data_dict[cytokine, condition, "fc_case_control"]}\t{data_dict[cytokine, condition, "pval"]}\t{data_dict[cytokine, condition, "cat_pvalue"]}\n")

output_txt.close()